In [1]:
import glacierml as gl

2022-07-12 12:43:22.659853: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
df = gl.data_loader()
df=df.drop('Zmed', axis=1)
df.name = 'df'
df

,CenLat,CenLon,Area,Zmin,Zmax,Slope,Aspect,Lmax,Thickness
0,67.91500,18.56400,1.438,1204,1698,16.9,98,1941,72.0
1,67.91000,18.49600,3.696,1207,2070,15.9,282,3954,84.0
2,67.90300,18.56900,3.405,1143,1797,13.2,102,3636,99.0
3,48.35698,-121.05735,2.924,1613,2196,12.8,350,3338,99.0
4,52.17540,-117.28400,16.154,1982,3448,12.6,93,10396,150.0
...,...,...,...,...,...,...,...,...,...
440,-3.05715,37.35070,0.616,5663,5794,9.9,355,905,23.0
441,77.02720,15.48540,0.418,323,587,18.8,144,1084,27.0
442,78.87100,12.16140,5.016,123,684,13.8,350,4111,76.0
443,46.21184,-121.49584,0.027,3396,3529,35.5,307,214,57.0


In [3]:
history_filename, model_filename = gl.build_and_train_model(df)

2022-07-12 12:43:25.796684: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-07-12 12:43:25.796705: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-07-12 12:43:25.797187: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


INFO:tensorflow:Assets written to: saved_models/sm/sm_10-5/df_1_dnn_MULTI_0.001_0.2_100_0/assets


In [4]:
import tensorflow as tf
model = tf.keras.models.load_model(model_filename)

In [5]:
train_features, test_features, train_labels, test_labels = gl.data_splitter(df)

In [6]:
import numpy as np
y_pred = model.predict(test_features).flatten()
y_true=test_labels
loss = tf.keras.losses.mean_absolute_error(y_true, y_pred)
print(loss)

3/3 [==============================] - 0s 1ms/step
tf.Tensor(30.323832, shape=(), dtype=float32)


In [7]:
import pandas as pd
model_history = pd.read_csv(history_filename)
model_history

,Unnamed: 0,loss,val_loss
0,0,66.271072,59.425335
1,1,66.202820,59.368404
2,2,66.129097,59.307529
3,3,66.051056,59.239178
4,4,65.962891,59.162956
...,...,...,...
95,95,26.433384,23.315111
96,96,26.400795,23.273481
97,97,26.340824,23.250265
98,98,26.288610,23.227413


In [8]:
RGI_All = gl.RGI_loader()
# RGI_All = RGI_All.reset_index()
RGI_All

100%|██████████| 19/19 [00:00<00:00, 30.96it/s]


,CenLat,CenLon,Slope,Zmin,Zmed,Zmax,Area,Aspect,Lmax
0,63.6890,-146.8230,42.0,1936,2385,2725,0.360,346,839
1,63.4040,-146.6680,16.0,1713,2005,2144,0.558,162,1197
2,63.3760,-146.0800,18.0,1609,1868,2182,1.685,175,2106
3,63.3810,-146.1200,19.0,1273,1944,2317,3.681,195,4175
4,63.5510,-147.0570,16.0,1494,1914,2317,2.573,181,2981
...,...,...,...,...,...,...,...,...,...
216496,-53.9860,-37.7325,29.9,310,-999,510,0.042,315,255
216497,-54.8310,-36.1361,23.6,330,-999,830,0.567,200,1130
216498,-54.1884,-37.3018,16.8,10,-999,1110,4.118,308,4329
216499,-68.8656,-90.4266,0.4,170,-999,270,0.011,122,106


In [ ]:
RGI = RGI.drop(RGI_All.loc[RGI_All['Zmed']<0].index)
RGI = RGI.drop(RGI.loc[RGI['Lmax']<0].index)
RGI = RGI.drop(RGI.loc[RGI['Slope']<0].index)
RGI = RGI.drop(RGI.loc[RGI['Aspect']<0].index)
RGI = RGI.reset_index()
RGI = RGI.drop('index', axis=1)
RGI = RGI.drop('Zmed', axis=1)
RGI

In [ ]:
y_pred = model.predict(RGI).flatten()

In [ ]:
np.mean(y_pred)

In [ ]:
print(f'Zmed missing: {len(RGI_All[RGI_All.Zmed<0])}\n'
      f'Slope missing: {len(RGI_All[RGI_All.Slope<0])}\n'
      f'Aspect missing: {len(RGI_All[RGI_All.Aspect<0])}\n'
      f'Lmax missing: {len(RGI_All[RGI_All.Lmax<0])}\n')

In [ ]:
print(f'Length w/o Zmed: {np.mean(RGI_All[RGI_All.Zmed<0].Lmax)}\n'
      f'Area w/ Zmed: {np.mean(RGI_All[RGI_All.Zmed>0].Area)}\n\n'
      f'Area w/o Slope: {np.mean(RGI_All[RGI_All.Slope<0].Area)}\n'
      f'Area w/ Slope: {np.mean(RGI_All[RGI_All.Slope>0].Area)}\n\n'
      f'Area w/o Aspect: {np.mean(RGI_All[RGI_All.Aspect<0].Area)}\n'
      f'Area w/ Aspect: {np.mean(RGI_All[RGI_All.Aspect>0].Area)}\n\n'
      f'Area w/o Lmax: {np.mean(RGI_All[RGI_All.Lmax<0].Area)}\n'
      f'Area w/ Lmax: {np.mean(RGI_All[RGI_All.Lmax>0].Area)}\n\n')

In [31]:
RGI_All

,CenLat,CenLon,Slope,Zmin,Zmed,Zmax,Area,Aspect,Lmax
0,63.6890,-146.8230,42.0,1936,2385,2725,0.360,346,839
1,63.4040,-146.6680,16.0,1713,2005,2144,0.558,162,1197
2,63.3760,-146.0800,18.0,1609,1868,2182,1.685,175,2106
3,63.3810,-146.1200,19.0,1273,1944,2317,3.681,195,4175
4,63.5510,-147.0570,16.0,1494,1914,2317,2.573,181,2981
...,...,...,...,...,...,...,...,...,...
216496,-53.9860,-37.7325,29.9,310,-999,510,0.042,315,255
216497,-54.8310,-36.1361,23.6,330,-999,830,0.567,200,1130
216498,-54.1884,-37.3018,16.8,10,-999,1110,4.118,308,4329
216499,-68.8656,-90.4266,0.4,170,-999,270,0.011,122,106


In [36]:
tau = 100e3
rho = 1e3
g = 10
alpha = RGI_All.Slope
H = tau / rho / g / np.sin(alpha / np.pi * 180)
H_avg = np.nansum(H) / sum(~np.isnan (H))
H_avg

inf

In [34]:
?np.sin